<a href="https://colab.research.google.com/github/vbipin/aip/blob/master/graph_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#ref: https://algs4.cs.princeton.edu/41graph/

In [0]:
class Edge :
    def __init__(self, src, dest, weight=1) :
        self.src    = src
        self.dest   = dest
        self.weight = float(weight)
    #for printing
    def __repr__(self) :
        return str((self.src, self.dest, self.weight))
    
class Graph : 
    def __init__(self) :
        self.V=0
        self.E=0
        self.nodes = {} #{ index: adjlist }
    
    def adj(self, v) :
        return self.nodes.get(v, [])
    
    def add_vertex(self, v) : #v is the new vertex index
        if v not in self.nodes :
            self.V += 1
            self.nodes[v] = [] #empty adj list
            
    def add_edge(self, src, dest, weight=1 ) : #edge is a tuple of (src, dest [weight])
        self.E += 1  
        self.nodes[src].append(Edge(src, dest, weight))
        
    def __str__(self) :
        return str(self.nodes)

In [0]:
#recursive dfs      
def dfs(g, start, end, visited=None) : 
    """returns True if found a path from start to end, else False"""
    visited = visited or {} #only for the first time

    if start == end :
        return True

    visited[start] = True #we mark it as visited

    for edge in g.adj(start) :  #we look at all the nodes next to the current
        if edge.dest not in visited : #we search if we havent visted them yet
            if dfs(g, edge.dest, end, visited) :
                return True

    return False   

In [0]:
#Non recursive dfs with paths
#We use a list as stack        
def dfs_path(g, start, end) : 
    """returns path if found a path from start to end, else []"""
    marked = {}
    stack   = []

    stack.append((start, [start])) #we are queing the index and path
    marked[start] = True #we mark it as visited
    
    while stack : #not empty
        current, path = stack.pop() #NOTE: This is what makes it a stack
        if current == end :
            return path

        for edge in g.adj(current) :  #we look at all the nodes next to the current
            if edge.dest not in marked : #we search if we havent visted them yet
                stack.append( (edge.dest, path+[edge.dest]) )
                marked[edge.dest] = True #we mark it as visited

    return []

In [0]:
#In bfs we simply use a queue instead of a stack
def bfs_path(g, start, end) : 
    """returns path if found a path from start to end, else False"""
    marked = {}
    queue   = [] #use pop(0) to make it as a queue
    
    queue.append((start, [start])) #we are queing the index and path
    marked[start] = True #we mark it as visited
    
    while queue : #not empty
        current, path = queue.pop(0)  #NOTE: This is what makes it a queue
        if current == end :
            return path

        for edge in g.adj(current) :  #we look at all the nodes next to the current
            if edge.dest not in marked : #we search if we havent visted them yet
                queue.append( (edge.dest, path+[edge.dest]) )
                marked[edge.dest] = True #we mark it as visited

    return []

In [0]:
################################################################################

In [0]:
###### Now some convenient funtions to build the graphs
def directed_graph(g, edge_list) : 
    """take a stream of tuples (src, dest [weight]) and build a directed graph"""
    for src, dest, *weight in edge_list : #weight is optional
        g.add_vertex(src)
        g.add_vertex(dest)
        g.add_edge(src, dest, *weight) #add the edge one by one
    return g

def undirected_graph(g, edge_list ) : 
    """take a stream of tuples (src, dest [weight]) and build a undirected graph"""
    for src, dest, *weight in edge_list : #weight is optional
        g.add_vertex(src)
        g.add_vertex(dest)
        g.add_edge(src, dest, *weight)
        g.add_edge(dest, src, *weight) #reverse the edge, so undirected
    return g

In [0]:
################################################################################

In [9]:
#https://algs4.cs.princeton.edu/41graph/images/graph.png
#ref: https://stackoverflow.com/questions/32370281/how-to-embed-image-or-picture-in-jupyter-notebook-either-from-a-local-machine-o
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://algs4.cs.princeton.edu/41graph/images/graph.png")

In [0]:
#This is the same graph as in https://algs4.cs.princeton.edu/41graph/
tiny_graph = """
0 5
4 3
0 1
9 12
6 4
5 4
0 2
11 12
9 10
0 6
7 8
9 11
5 3
"""
def text_to_edges( lines ) :
    return [ [ int(i) for i in line.split()] for line in tiny_graph.split("\n") if line and len(line.split()) > 1 ]

g = undirected_graph( Graph(), text_to_edges( tiny_graph ) )

In [11]:
dfs_path(g,2,3)

[2, 0, 6, 4, 3]

In [12]:
bfs_path(g,2,3)

[2, 0, 5, 3]

In [0]:
################################################################################

In [0]:
#We create some sample search problems to check our algorithms
#this class will create a 2D grid of row x colums and its graph.
#Some of the cells can be disabled by putting it into walls
#for example SearchGrid(3,4,walls=[0,10]) will create a grid that has cell 0 and 10 are disabled.

class SearchGrid :
    def __init__(self, rows=5, columns=5, walls=[] ) : #walls -> [ list of index ]
        self.rows = rows
        self.columns = columns
        self.N = rows * columns #total cells
        self.walls = walls
        
    def edges(self) : 
        """return edges of the graph in a list of tuples (u,v)"""
        edges = []

        #Just a convenient funtion
        def _add_edge(u, v) :
            #we add the edge if source and destinations are not walls
            #and within the grid
            if u not in self.walls and v not in self.walls and u < self.N and v < self.N :
                edges.append( (u,v) )
        
        #forward links and downward links
        for i in range(self.N) :
            if (i+1) % self.columns != 0 :#checking if it is an edge cell
                _add_edge(i, i+1)         #connect to next cell
            _add_edge(i,i+self.columns)  #connect to the cell below; it is i+width
        
        return edges
    
    #pretty print the grid and path if given. path -> [ list of nodes ]
    def print(self, path=[]) :
        for i in range(self.N) :
            if i in self.walls :
                print('# ', end='')
            elif i in path :
                print('^ ', end='')
            else :
                print('. ', end='')
            if (i+1) % self.columns == 0 :
                print("")

In [0]:
grid = SearchGrid(3,4,walls=[10,2])
g = undirected_graph(Graph(), grid.edges() )

In [16]:
grid.print(bfs_path(g,11,9))

. . # . 
. ^ ^ ^ 
. ^ # ^ 


In [0]:
#Make a grid with walls
grid = SearchGrid(10,10,walls=[10,2,17,199])

#Make the graph of the grid from the edges
g = undirected_graph(Graph(), grid.edges() )

In [18]:
path = bfs_path(g,0,grid.N-1)
grid.print(path)

^ ^ # . . . . . . . 
# ^ ^ ^ ^ ^ ^ # . . 
. . . . . . ^ ^ ^ ^ 
. . . . . . . . . ^ 
. . . . . . . . . ^ 
. . . . . . . . . ^ 
. . . . . . . . . ^ 
. . . . . . . . . ^ 
. . . . . . . . . ^ 
. . . . . . . . . ^ 


In [19]:
path = dfs_path(g,0,grid.N-1)
grid.print(path)

^ ^ # . . . . . . . 
# ^ . . . . . # . . 
. ^ . . . . . . . . 
. ^ . . . . . . . . 
. ^ . . . . . . . . 
. ^ . . . . . . . . 
. ^ . . . . . . . . 
. ^ . . . . . . . . 
. ^ . . . . . . . . 
. ^ ^ ^ ^ ^ ^ ^ ^ ^ 
